In [4]:
from pathlib import Path

import h5py
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm_notebook
from minkowski import compute_minkowski
from utils import fix_random_seed, postprocess_cube, two_point_correlation

In [5]:
dataset_path = Path('../../data/berea_64/')
files = list(dataset_path.glob('*.hdf5'))

In [7]:
data = {
    'V': [],
    'S': [],
    'B': [],
    'Xi': []
}
for file in tqdm_notebook(files, desc="Sample"):
    with h5py.File(file, 'r') as fl:
        cube = fl['data'][()]
    cube = cube // 255
    cube = np.pad(cube, ((1, 1), (1, 1), (1, 1)), mode='constant', constant_values=0)
    v, s, b, xi = compute_minkowski(cube)
    data['V'].append(v)
    data['S'].append(s)
    data['B'].append(b)
    data['Xi'].append(xi)

    two_point_covariance = {}
    grain_value = cube.max()
    for i, direct in enumerate(["x", "y", "z"]):
        two_point_direct = two_point_correlation(cube, i, var=grain_value)
        two_point_covariance[direct] = two_point_direct
    # phase averaging
    direct_covariances = {}
    for direct in ["x", "y", "z"]:
        direct_covariances[direct] = np.mean(np.mean(two_point_covariance[direct], axis=0), axis=0)
    # covariance storage
    covariance_df = pd.DataFrame(direct_covariances)
    covariance_df.to_csv(dataset_path / 'covariance_stats' / (file.name + ".csv"), index=False)

df = pd.DataFrame(data)
df.to_csv(dataset_path / 'minkowski64.csv', index=False)